In [2]:
%pip install requests beautifulsoup4 lxml 


     ---------------------------------------- 3.8/3.8 MB 11.6 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [6]:
def scrape_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
        }
        # Send a GET request with headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'lxml')

        # Extract meaningful text (e.g., from <p> tags)
        paragraphs = soup.find_all('p')
        text = '\n'.join(paragraph.get_text(strip=True) for paragraph in paragraphs)

        return text if text else "No meaningful text found on this page."

    except requests.exceptions.RequestException as e:
        return f"An error occurred while fetching the URL: {e}"

In [98]:
def clean_text(text, stop_line):
    """
    Remove a specific line and everything after it from the text.
    
    Args:
        text (str): The full extracted text.
        stop_line (str): The line indicating where to stop.
        
    Returns:
        str: The cleaned text.
    """
    lines = text.split('\n')  # Split the text into lines
    cleaned_lines = []

    year_pattern = re.compile(r"^(\d{4})—(.+)$")
    
    for line in lines:
        if stop_line in line:
            break  # Stop processing once the stop line is found
        cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines)  # Rejoin the cleaned lines

In [119]:
def process_text(input_text):
    """
    Extract lines starting with a year in various formats and format them as a dictionary.

    Args:
        input_text (str): Raw text input.

    Returns:
        dict: A dictionary with years as keys and the text after the separator as values.
    """
    result = {}

    # Unified regex to handle multiple patterns
    year_pattern = re.compile(r"^(\d{4})\s*[—–-]{1,2}\s*(.+)$")


    # Split the text into lines and process each
    lines = input_text.splitlines()
    for line in lines:
        match = year_pattern.match(line)
        if match:
            year = match.group(1)  # Extract the year
            text = match.group(2).strip()  # Extract and clean the text after the year
            result[year] = text

    return result

In [20]:
def filter_years(data, selected_years):
    """
    Filter the dictionary to only keep selected years.

    Args:
        data (dict): The full dictionary of years and their text.
        selected_years (list): List of years to keep.

    Returns:
        dict: Filtered dictionary with only the selected years.
    """
    return {year: data[year] for year in selected_years if year in data}

In [135]:
url = 'https://triblive.com/sports/this-date-in-sports-history-jan-20/'

extracted_text = scrape_url(url)
# print(extracted_text)

clean_text_ = clean_text(extracted_text, "Remove the adsfrom your TribLIVE reading experience")

clean_dict = process_text(clean_text_)

print(f"COPY: {url}")
display(clean_dict)

COPY: https://triblive.com/sports/this-date-in-sports-history-jan-20/


{'1891': 'The International YMCA in Springfield, Mass., is the site of the first official basketball game. Peach baskets were used, but it wasn’t until 1905 that someone removed the baskets’ bottoms.',
 '1937': 'Nels Stewart of the New York Americans becomes the NHL’s all-time scorer with his 270th goal in a 4-0 victory over the Montreal Canadiens.',
 '1952': 'George Mikan scores 61 points, a career-high, to lead the Minneapolis Lakers to a 91-81 double-overtime victory over the Rochester Royals.',
 '1966': 'Ted Williams, longtime star of the Boston Red Sox, is elected to the Baseball Hall of Fame. Williams was a two-time triple crown winner, a two-time MVP and the last player to hit over .400 — despite losing five prime years to service in the Marine Corps.',
 '1968': 'Elvin Hayes scores 39 points to lead Houston to a 71-69 victory and end UCLA’s 47-game winning streak. A regular-season record 52,693 fans attend the game at the Houston Astrodome.',
 '1970': 'Cincinnati’s Tom Van Arsda

In [136]:
selected_years = ['', '', '']
selected_years = ['', '', '']
selected_years = ['', '', '']
selected_years = ['', '', '']
selected_years = ['1891', '1980', '2008']
final_dict = filter_years(clean_dict, selected_years)

for key in final_dict.keys():
    final_dict[key] = re.sub("\\u201c", "'", final_dict[key])
    final_dict[key] = re.sub("\\u201d", "'", final_dict[key])
    final_dict[key] = re.sub("\\u2019", "'", final_dict[key])

formatted_output = json.dumps(final_dict, indent=4)

print(formatted_output)

{
    "1891": "The International YMCA in Springfield, Mass., is the site of the first official basketball game. Peach baskets were used, but it wasn't until 1905 that someone removed the baskets' bottoms.",
    "1980": "Terry Bradshaw passes for 309 yards and sets two passing records to help the Pittsburgh Steelers beat the Los Angeles Rams 31-19 and become the first team to win four Super Bowls.",
    "2008": "Lawrence Tynes, who missed a 36-yard field goal at the end of regulation following a bad snap, nails a 47-yarder in overtime to give the New York Giants a 23-20 win over Green Bay in the NFC championship at Lambeau Field. The Giants win their 10th straight on the road in the third-coldest championship game ever."
}
